# Decision Tree Regressor with Pyspark's Structured API

Importing necessary packages

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, udf
from math import radians, sin, cos, sqrt, atan2
from pyspark.sql.types import DoubleType

## 1. Preparation

Let's initialize a Spark session:

In [2]:
#pyspark init
builder = SparkSession.builder\
            .appName('taxi_duration_highAPI')\
            .config("spark.driver.memory", "4g")\
            .config("spark.executor.memory", "4g")
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/12 20:41:24 WARN Utils: Your hostname, HP-Envy resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/12 20:41:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 20:41:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/12 20:41:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Read input datasets for training and testing:

In [3]:
#Read input files
raw_train_data = spark.read.csv('train.csv', header=True, inferSchema=True)
raw_test_data = spark.read.csv('test.csv', header=True, inferSchema=True)

## 2. Data preprocessing

Parse timestamp features in the dataset:

In [4]:
#Cast the pickup string values of training data into timestamps.
casted_train_data = raw_train_data.withColumns({
                        'pickup_datetime' : raw_train_data['pickup_datetime'].cast('timestamp'),
                    })

#Cast the pickup string values of testing data into timestamps.
casted_test_data = raw_test_data.withColumns({
                        'pickup_datetime' : raw_test_data['pickup_datetime'].cast('timestamp'),
                    })

User-Defined Function (UDF)

**Tính Khoảng Cách Haversine**

The `haversine` formula is used to calculate the **geographical distance** between two points on the Earth's surface based on their longitude and latitude coordinates. This feature is added to the taxi trip duration prediction problem (`trip_duration`), as the travel time often depends on the actual distance between the pickup and drop-off locations.

**Haversine Formula**

The Haversine formula determines the distance between two points on a sphere.

Assumed that the Earth is a sphere with the mean radius $ R = 6371 \, \text{km} $, the distance between 2 locations (points) can be calculated as:

$$
a = \sin^2\left(\frac{\Delta \text{lat}}{2}\right) + \cos(\text{lat}_1) \cdot \cos(\text{lat}_2) \cdot \sin^2\left(\frac{\Delta \text{lon}}{2}\right)
$$ 

$$
\theta = 2 \cdot \text{atan2}\left(\sqrt{a}, \sqrt{1-a}\right)
$$

$$
d = R \cdot \theta
$$

Trong đó:
- $ \text{lon}_1, \text{lat}_1 $: Longitude and latitude of the pickup point.
- $ \text{lon}_2, \text{lat}_2 $: Longitude and latitude of the dropoff point.
- $ \Delta \text{lon} = \text{lon}_2 - \text{lon}_1 $: Longitude difference.
- $ \Delta \text{lat} = \text{lat}_2 - \text{lat}_1 $: Latitude difference.
- $ R = 6371 \, \text{km} $: Earth's radius.
- $ d $: Distance between pickup and dropoff points (in kilometer).
- $ a $: The "haversine" representation of the central angle between two points.
- $ \theta $: The central angle (in radian) between two points on the sphere. 
- $ d $: Multiply the central angle with Earth's radius to get the distance between pickup and dropoff points.

In [5]:
# Haversine distance function
def haversine(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2) or not all(isinstance(x, (int, float)) for x in [lon1, lat1, lon2, lat2]):
        return 0.0
    R = 6371  # Earth's radius (km)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

haversine_udf = udf(haversine, DoubleType())

Extract usable features:

In [6]:
#Get usable columns from the dataframe
#Also convert timestamps into time elements and encode `store_and_fwd_flag` feature into binary values
extracted_train_df = casted_train_data.selectExpr(
    'id',
    'vendor_id',
    'YEAR(pickup_datetime) AS pickup_year',
    'MONTH(pickup_datetime) AS pickup_month',
    'DAY(pickup_datetime) AS pickup_day',
    'HOUR(pickup_datetime) AS pickup_hour',
    'MINUTE(pickup_datetime) AS pickup_min',
    'SECOND(pickup_datetime) AS pickup_sec',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'CASE WHEN store_and_fwd_flag = "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag',
    'trip_duration'
).withColumn('distance_km', haversine_udf(col('pickup_longitude'), col('pickup_latitude'), col('dropoff_longitude'), col('dropoff_latitude')))

extracted_test_df = casted_test_data.selectExpr(
    'id',
    'vendor_id',
    'YEAR(pickup_datetime) AS pickup_year',
    'MONTH(pickup_datetime) AS pickup_month',
    'DAY(pickup_datetime) AS pickup_day',
    'HOUR(pickup_datetime) AS pickup_hour',
    'MINUTE(pickup_datetime) AS pickup_min',
    'SECOND(pickup_datetime) AS pickup_sec',
    'passenger_count',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'CASE WHEN store_and_fwd_flag = "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag'
).withColumn('distance_km', haversine_udf(col('pickup_longitude'), col('pickup_latitude'), col('dropoff_longitude'), col('dropoff_latitude')))

In [7]:
# Removing outliers 
extracted_train_df = extracted_train_df.filter(
    (col('trip_duration') > 0) & 
    (col('trip_duration') < 36000) &  # Limited to 10 hours
    (col('passenger_count') >= 1) & 
    (col('passenger_count') <= 6) &  # Passenger limited
    (col('distance_km') > 0) & 
    (col('distance_km') < 100)  # Distance limited to 100km
)

# null value checking
print("Null value checking in extracted_train_df:")
for column in extracted_train_df.columns:
    null_count = extracted_train_df.filter(col(column).isNull()).count()
    print(f"{column}: {null_count} null value(s)")

extracted_train_df = extracted_train_df.na.drop()

Null value checking in extracted_train_df:


id: 0 null value(s)


vendor_id: 0 null value(s)


pickup_year: 0 null value(s)


pickup_month: 0 null value(s)


...

pickup_latitude: 0 null value(s)


dropoff_longitude: 0 null value(s)


dropoff_latitude: 0 null value(s)
store_and_fwd_flag: 0 null value(s)


trip_duration: 0 null value(s)


distance_km: 0 null value(s)


Encode the features into vector of values:

In [8]:
#Extract training features name

# List of features (excluding 'id' and 'trip_duration')
feature_cols = ['vendor_id', 'pickup_year', 'pickup_month', 'pickup_day', 'pickup_hour',
                'pickup_min', 'pickup_sec', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
                'store_and_fwd_flag', 'distance_km']

#Use VectorAssembler to transform feature columns into a single vector
extracted_train_df.printSchema()
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

# Convert training data
train_data = assembler.transform(extracted_train_df).select('id', 'features', 'trip_duration')

root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_min: integer (nullable = true)
 |-- pickup_sec: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: integer (nullable = false)
 |-- trip_duration: integer (nullable = true)
 |-- distance_km: double (nullable = true)



## 3. Model training

Split data

In [ ]:
#Split data into training and testing sets
#Training set proportion parameter:
train_size = 0.8

train, validation = train_data.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

Fine-tuning

In [10]:
# Fine-tuning
param_grid = {
    'maxDepth': [5, 10, len(feature_cols) ] # try max_depth = 5, 10, and the number of features used (14)
}

best_rmse = float('inf')
best_params = None
best_model = None

estimator = DecisionTreeRegressor(
        featuresCol='features',
        labelCol='trip_duration',
    )

for maxDepth in param_grid['maxDepth']:
    estimator = estimator.setMaxDepth(maxDepth)
    model = estimator.fit(train)
    
    predictions = model.transform(validation)
    evaluator_rmse = RegressionEvaluator(labelCol='trip_duration', predictionCol='prediction', metricName='rmse')
    rmse = evaluator_rmse.evaluate(predictions)
    print(f"Params: maxDepth={maxDepth}, RMSE={rmse}")
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_params = {'maxDepth': maxDepth}
        best_model = model

Params: maxDepth=5, RMSE=416.17375300265064


Params: maxDepth=10, RMSE=389.25328786776396


25/04/12 20:44:13 WARN DAGScheduler: Broadcasting large task binary with size 1201.2 KiB
25/04/12 20:44:14 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
25/04/12 20:44:16 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


Params: maxDepth=14, RMSE=408.58052592870024


## 4. Model evaluation (hold-out)

In [11]:
# Perform evaluation on the best parameters configuration found
predictions = best_model.transform(validation)
evaluator_rmse = RegressionEvaluator(labelCol='trip_duration', predictionCol='prediction', metricName='rmse')
evaluator_r2 = RegressionEvaluator(labelCol='trip_duration', predictionCol='prediction', metricName='r2')
rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"Best Params: {best_params}")
print(f"Best RMSE on validation set: {rmse}")
print(f"Best R2 on validation set: {r2}")
print("Feature Importances:", best_model.featureImportances)
print("Tree structure:")
print(best_model.toDebugString)

Best Params: {'maxDepth': 10}
Best RMSE on validation set: 389.25328786776396
Best R2 on validation set: 0.672272085049675
Feature Importances: (14,[0,2,3,4,5,6,7,8,9,10,11,12,13],[3.5712358387544006e-05,0.005694488132320071,0.0011410693818057534,0.08271983268506268,0.00108825535536416,0.00017532566643481176,1.9210847318708636e-05,0.008097165512724686,0.0071135907846128195,0.00895301975408897,0.023315617670300763,3.7196306774956215e-05,0.861609515544804])
Tree structure:
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_ec37f347339a, depth=10, numNodes=2011, numFeatures=14
  If (feature 13 <= 4.324768240904582)
   If (feature 13 <= 1.864440183564458)
    If (feature 13 <= 1.1364797472169215)
     If (feature 13 <= 0.8709855080686648)
      If (feature 11 <= 40.76713752746582)
       If (feature 4 <= 7.5)
        If (feature 13 <= 0.768386976707994)
         If (feature 7 <= 2.5)
          If (feature 4 <= 1.5)
           If (feature 10 <= -73.98360061645508)
            Predict: 2

## 5. Prediction (test file)

In [12]:
# Perform prediction on testing set
test_data = assembler.transform(extracted_test_df).select('id', 'features')
test_predictions = model.transform(test_data).select('id', 'prediction').withColumnRenamed('prediction', 'trip_duration')

# Write file
test_predictions.coalesce(1).write.csv('prediction_highAPI.csv', header=True, mode='overwrite')

# Display some records of prediction
test_predictions.show(5)

# Close SparkSession
spark.stop()

25/04/12 20:44:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/04/12 20:44:38 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
25/04/12 20:44:43 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+---------+------------------+
|       id|     trip_duration|
+---------+------------------+
|id3004672| 783.7178051511759|
|id3505355| 616.5298913043479|
|id1217141| 453.3487824037706|
|id2150126| 1247.785243198681|
|id1598245|414.41267123287673|
+---------+------------------+
only showing top 5 rows

